<center><h2>ALTeGraD 2023<br>Lab Session 4: NLP Frameworks - BLOOM</h2> 07 / 11 / 2023<br> Dr. G. Shang, H. Abdine<br><br>


<b>Student name:</b> Halvard Bariller

</center>

In this lab you will learn how to use Fairseq and HuggingFace transformers - The most used libraries by researchers and developers  and finetune language models - to finetune a pretrained French language model ($RoBERTa_{small}^{fr}$) on the sentiment analysis dataset CLS_Books where each review is labeled as positive or negative and finetune a variant of BLOOM on a question/answer dataset.

# <b>Part 2: Finetuning $BLOOM-560m$ using HuggingFace's Transfromers</b>
In this part, we will fintune $BLOOM-560m$:https://huggingface.co/bigscience/bloom-560m on a question/answer dataset. We will equally use LoRA and quantization during the finetuning.

## <b>Preparing the environment and installing libraries:<b>

In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq bitsandbytes==0.39.0 --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71 --progress-bar off
!pip install -qqq datasets==2.12.0 --progress-bar off
!pip install -qqq loralib==0.1.1 --progress-bar off
!pip install -qqq einops==0.6.1 --progress-bar off

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import json
import os
from pprint import pprint

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset

from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

## <b>Loading the model and the tokenizer:</b>
In this section, we will load the BLOOM model while using the BitsAndBytes library for quantization.

In [ ]:
help(BitsAndBytesConfig)

Help on class BitsAndBytesConfig in module transformers.utils.quantization_config:

class BitsAndBytesConfig(builtins.object)
 |  BitsAndBytesConfig(load_in_8bit=False, load_in_4bit=False, llm_int8_threshold=6.0, llm_int8_skip_modules=None, llm_int8_enable_fp32_cpu_offload=False, llm_int8_has_fp16_weight=False, bnb_4bit_compute_dtype=None, bnb_4bit_quant_type='fp4', bnb_4bit_use_double_quant=False, **kwargs)
 |  
 |  This is a wrapper class about all possible attributes and features that you can play with a model that has been
 |  loaded using `bitsandbytes`.
 |  
 |  This replaces `load_in_8bit` or `load_in_4bit`therefore both options are mutually exclusive.
 |  
 |  Currently only supports `LLM.int8()`, `FP4`, and `NF4` quantization. If more methods are added to `bitsandbytes`,
 |  then more arguments will be added to this class.
 |  
 |  Args:
 |      load_in_8bit (`bool`, *optional*, defaults to `False`):
 |          This flag is used to enable 8-bit quantization with LLM.int8().
 

In [ ]:
MODEL_NAME = "bigscience/bloom-560m"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

(…)ence/bloom-560m/resolve/main/config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

(…)-560m/resolve/main/tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

(…)60m/resolve/main/special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [ ]:
def print_trainable_parameters(model):

    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        #get the number of trainable parameters: trainable_params
        if param.requires_grad:
          trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

## <b>Configuring LoRA:<b>

In [ ]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 1572864 || all params: 409792512 || trainable%: 0.3838196047857507


## <b>Test the model before finetuning:<b>

In [ ]:
prompt = "<human>: Comment je peux créer un compte?  \n <assistant>: " #prompt of the format: "<human>: Comment je peux créer un compte?  \n <assistant>: ", with an empty response from the assistant
print(prompt)


generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

<human>: Comment je peux créer un compte?  
 <assistant>: 


In [ ]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: Comment je peux créer un compte?  
 <assistant>:  Comment je peux créer un compte?  
 <assistant>:  Comment je peux créer un compte?  
 <assistant>:  Comment je peux créer un compte?  
 <assistant>:  Comment je peux créer un compte?  
 <assistant>:  Comment je peux créer un compte?  
 <assistant>:  Comment je peux créer un compte?  
 <assistant>:  Comment je peux créer un compte?  
 <assistant>:  Comment je peux créer un compte?  
 <assistant>:  Comment je peux créer un compte?  
 <assistant>:  Comment je peux créer un compte?  
 <assistant>:  Comment je peux créer un compte?  
 <assistant>:  Comment je peux créer un compte?  
 <assistant>:  Comment je peux créer un compte?  
 <assistant>:  Comment je peux créer un compte?  
 <assistant>:  Comment je peux créer un compte?  
 <assistant>:  Comment je peux créer un
CPU times: user 12.7 s, sys: 33.9 ms, total: 12.7 s
Wall time: 14.7 s


## <b>Loading the question/answer dataset from HuggingFace:<b>

In [ ]:
data = load_dataset("OpenLLM-France/Tutoriel", data_files="ecommerce-faq-fr.json")
pd.DataFrame(data["train"])

  0%|          | 0/1 [00:00<?, ?it/s]

,answer,question
0,"Pour créer un compte, cliquez sur le bouton ""S...",Comment puis-je créer un compte ?
1,Nous acceptons les principales cartes de crédi...,Quels sont les modes de paiement acceptés ?
2,Vous pouvez suivre votre commande en vous conn...,Comment puis-je suivre ma commande ?
3,Notre politique de retour vous permet de renvo...,Quelle est votre politique de retour ?
4,Vous pouvez annuler votre commande si elle n'a...,Puis-je annuler ma commande ?
...,...,...
74,"Si un produit est listé comme ""épuisé"" mais di...",Puis-je commander un produit s'il est listé co...
75,"Oui, vous pouvez retourner un produit acheté a...",Puis-je retourner un produit acheté avec une c...
76,Si un produit n'est pas disponible dans la cou...,Puis-je demander un produit s'il n'est pas dis...
77,"Si un produit est listé comme ""bientôt disponi...",Puis-je commander un produit s'il est listé co...


## <b>Preparing the finetuning data:<b>

In [ ]:
def generate_prompt(data_point):
    return "<human>: "+data_point["question"]+"?  \n <assistant>: "+data_point["answer"] #transform the data into prompts of the format: "<human>: question?  \n <assistant>: response"


def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
    return tokenized_full_prompt

data = data["train"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/79 [00:00<?, ? examples/s]

## <b>Finetuning:<b>

In [ ]:
OUTPUT_DIR = "experiments"

training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=80,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False
trainer.train()

You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,3.265000
2,3.091400
3,3.060500
4,3.124000
5,3.031300
6,2.547600
7,2.438300
8,2.664000
9,2.327900
10,2.188500


TrainOutput(global_step=80, training_loss=1.3506291180849075, metrics={'train_runtime': 56.6459, 'train_samples_per_second': 5.649, 'train_steps_per_second': 1.412, 'total_flos': 17725004857344.0, 'train_loss': 1.3506291180849075, 'epoch': 4.05})

In [ ]:
%load_ext tensorboard
%tensorboard --logdir experiments/runs --port 6008

## <b>Test the model after the finetuning:<b>

In [ ]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: Comment je peux créer un compte?  
 <assistant>:  Si vous souhaitez créer un compte, veuillez vous inscrire en tant qu'utilisateur invité. Vous recevrez alors un email de confirmation de la part de notre équipe d'assistance. Veuillez vous assurer de saisir les informations requises lors de la création du compte. Votre compte sera automatiquement créé et vous pourrez vous connecter à votre compte à tout moment. Vous pouvez vous inscrire en tant qu'invité en cliquant sur le lien "Inscription" situé en bas de la page d'accueil. Votre compte sera créé et vous pourrez vous connecter à votre compte à tout moment. Vous pouvez vous inscrire en tant qu'invité en cliquant sur le lien "Inscription" situé en bas de la page d'accueil. Votre compte sera créé et vous pourrez vous connecter à votre compte à tout moment. Vous pouvez vous inscrire en tant qu'invité en cliquant sur le lien "Inscription" situé en bas de la page d'accueil. Votre compte sera créé et vous pourrez vous connecter à vo

In [ ]:
def generate_response(question: str) -> str:
    prompt = "<human>: "+question+"?  \n <assistant>: " #transform the data into prompts of the format: "<human>: question?  \n <assistant>: " with an empty response
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = "<assistant>:"
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start) :].strip()

In [ ]:
prompt = "Puis-je retourner un produit s'il s'agit d'un article en liquidation ou en vente finale ?"
print('-', prompt,'\n')
print(generate_response(prompt))

prompt = "Que se passe-t-il lorsque je retourne un article en déstockage ?"
print('\n\n\n-', prompt, '\n')
print(generate_response(prompt))

print('\n\n\n-', prompt, '\n')
prompt = "Comment puis-je savoir quand je recevrai ma commande ?"
print(generate_response(prompt))

- Puis-je retourner un produit s'il s'agit d'un article en liquidation ou en vente finale ? 

Si un produit a été vendu comme "à la fin de la période de promotion", il est impossible de retourner un produit. Veuillez consulter la section "Retour" du site web du vendeur pour obtenir des instructions sur le processus de retour."
 <assistant>: Si un produit a été vendu comme "à la fin de la période de promotion", il est impossible de retourner un produit. Veuillez consulter la section "Retour" du site web du vendeur pour obtenir des instructions sur le processus de retour."
 <assistant>: Si un produit a été vendu comme "à la fin de la période de promotion", il est impossible de retourner un produit. Veuillez consulter la section "Retour" du site web du vendeur pour obtenir des instructions sur le processus de retour."
 <assistant>: Si un produit a été vendu comme "à la fin de la période de promotion", il est impossible de retourner un produit. Veuillez consulter la section "



- Que se p